In [1]:
import os, anparse
an_file = 'blc/Laws.an'
laws = list(anparse.tf_anfile('BLC', an_file))

In [2]:
# for word in laws:
#     print(word)
tsv='\n'.join((word for word in laws))

In [3]:
print(tsv)

BLC	0	1	TWB	TWB	TWB	TWB	trailer= 	lex=TWB	sp=advb
BLC	0	1	KTB=/~>	KTB=>	KTB=>	KTB>	trailer= 	lex=KTB=	nme=	emf=>	sp=subs	nu=NA	gn=m	st=emph
BLC	0	1	D	D	D	D	trailer=	lex=D	sp=prep	ls=pcon
BLC	0	1	NMWS/(J~>	NMWS(J>	NMWSJ>	NMWS>	trailer= 	lex=NMWS	nme=J	emf=>	sp=subs	nu=pl	gn=m	st=emph
BLC	0	1	D	D	D	D	trailer=	lex=D	sp=prep	ls=pcon
BLC	0	1	>TR/&WT=~>	>TR&WT=>	>TRT=>	>TRWT>	trailer= 	lex=>TR	nme=T=	emf=>	sp=subs	nu=pl	gn=m	st=emph
BLC	1	1	MN	MN	MN	MN	trailer= 	lex=MN	sp=prep
BLC	1	1	QDM	QDM	QDM	QDM	trailer= 	lex=QDM	sp=prep
BLC	1	1	JWM/T=~>	JWMT=>	JWMT=>	JWMT>	trailer= 	lex=JWM	nme=T=	emf=>	sp=subs	nu=pl	gn=m	st=emph
BLC	1	1	<L=[/JN	<L=JN	<L=JN	<LJN	trailer= 	lex=<L=	vbe=	nme=JN	sp=verb	nu=pl	gn=m	st=abs	vo=act	vs=pe	vt=ptc
BLC	1	1	HWJ[N	HWJN	HWJN	HWJN	trailer= 	lex=HWJ	vbe=N	nu=pl	sp=verb	vo=act	vs=pe	vt=pf	ps=first	ls=vbex
BLC	1	1	L	L	L	L	trailer=	lex=L	sp=prep
BLC	1	1	!M!S<R=[/	MS<R=	MS<R=	MS<R	trailer= 	pfm=M	lex=S<R=	vbe=	nme=	sp=verb	st=NA	vo=act	vs=pe	vt=inf
BLC	1	1	L	L	L	L	trailer=

In [7]:
with open('../../data/blc/laws.tsv', 'w') as f:
    f.write(tsv)

In [8]:
def maketrans(s1, s2):
    return dict(zip([ord(a) for a in s1], [ord(a) for a in s2]))

tosyr=maketrans('>BGDHWZXVJKLMNS<PYQRCT','ܐܒܓܕܗܘܙܚܛܝܟܠܡܢܣܥܦܨܩܪܫܬ')

In [29]:
import collections


commonMetaData = collections.OrderedDict(
    dataset='BLC',
    datasetName='Book of the Laws of the Countries',
    source='ETCBC',
    sourceUrl='',
    encoders=('Dirk Bakker'
              'Hannes Vlaardingerbroek (TF)'),
    email1='hannes@vlaardingerbroek.nl',
)
specificMetaData = collections.OrderedDict(
    title='book name',
    number='verse/chapter number',
    word='full form of the word in syriac script',
    word_etcbc='full form of the word in ETCBC transcription',
)
langMetaData = dict(
    en=dict(
        language='English',
        languageCode='en',
        languageEnglish='English',
    ),
)
numFeatures = set(
    '''
    number
'''.strip().split()
)

oText = {
    'sectionFeatures': 'book,chapter,verse',
    'sectionTypes': 'book,chapter,verse',
    'fmt:text-orig-full': '{word} ',
    'fmt:text-trans-full': '{word_ascii} ',
    'fmt:lex-orig-full': '{lexeme} ',
    'fmt:lex-trans-full': '{lexeme_ascii} ',
}



cur = collections.Counter()
curSlot = 0
context = []

nodeFeatures = collections.defaultdict(dict)
edgeFeatures = collections.defaultdict(
    lambda: collections.defaultdict(set)
)

oSlots = collections.defaultdict(set)
(prevBook, prevChapter, prevVerse) = (None, None, None)

lexemes = set()


with open('../../data/blc/laws.tsv') as f:
    for line in f:
        book, chapter, verse,_,_,_, surface, *features = line.rstrip('\n').split('\t')
        wordfeatures = {}
        for f in features:
#             print(f)
            k,v = f.split('=',1)
            wordfeatures[k]=v
        lexeme = wordfeatures['lex']

        if book != prevBook:
        
            if prevVerse is not None:
                context.pop()
                prevVerse = None
            if prevChapter is not None:
                context.pop()
                prevChapter = None
            if prevBook is not None:
                context.pop()
                
            cur['book'] += 1
            prevBook = book
            bookNode = ('book', cur['book'])
            nodeFeatures['title'][bookNode] = book

            context.append(bookNode)
        
        if chapter != prevChapter:
        
            if prevVerse is not None:
                context.pop()
                prevVerse = None
            if prevChapter is not None:
                context.pop()
                prevChapter = None
                
            cur['chapter'] += 1
            prevChapter = chapter
            chapterNode = ('chapter', cur['chapter'])
            nodeFeatures['number'][chapterNode] = chapter
            context.append(chapterNode)
        
        if verse != prevVerse:
        
            if prevVerse is not None:
                context.pop()
                prevVerse = None
                
            cur['verse'] += 1
            prevVerse = verse
            verseNode = ('verse', cur['verse'])
            nodeFeatures['number'][verseNode] = verse
            context.append(verseNode)

        curSlot += 1
        wordNode = ('word', curSlot)
        
        nodeFeatures['word'][wordNode] = surface.translate(tosyr)
        nodeFeatures['word_etcbc'][wordNode] = surface
        
        for k,v in wordfeatures.items():
            nodeFeatures[k][wordNode] = v
        
        if lexeme not in lexemes:
            lexemes.add(lexeme)
            cur['lex'] += 1
            lexNode = ('lex', cur['lex'])
            nodeFeatures['lex'][lexNode] = lexeme

        context.append(('lex', cur['lex']))
        for (nt, curNode) in context:
            oSlots[(nt, curNode)].add(curSlot)

        context.pop()
    context.pop()
    context.pop()
    context.pop()
    
if len(context):
    print('Context:', context)

print(f'\n{curSlot:>7} x slot')
for (nodeType, amount) in sorted(cur.items(), key=lambda x: (x[1], x[0])):
    print(f'{amount:>7} x {nodeType}')

    
print('Compiling TF data')
print(f'Building warp feature otype')
nodeOffset = {'word': 0}
oType = {}
n = 1
for k in range(n, curSlot + 1):
    oType[k] = 'word'
n = curSlot + 1
for (nodeType, amount) in sorted(cur.items(), key=lambda x: (x[1], x[0])):
    nodeOffset[nodeType] = n - 1
    for k in range(n, n + amount):
        oType[k] = nodeType
    n = n + amount
print(f'{len(oType)} nodes')

print('Filling in the nodes for features')
newNodeFeatures = collections.defaultdict(dict)
for (ft, featureData) in nodeFeatures.items():
    newFeatureData = {}
#     print(featureData)
    for ((nodeType, node), value) in featureData.items():
        newFeatureData[nodeOffset[nodeType] + node] = value
    newNodeFeatures[ft] = newFeatureData
newOslots = {}
for ((nodeType, node), slots) in oSlots.items():
    newOslots[nodeOffset[nodeType] + node] = slots

nodeFeatures = newNodeFeatures
nodeFeatures['otype'] = oType
edgeFeatures['oslots'] = newOslots

nodeFeatures['book@en'] = nodeFeatures['book']

print(f'Node features: {" ".join(nodeFeatures)}')
print(f'Edge features: {" ".join(edgeFeatures)}')


metaData = {
    '': commonMetaData,
    'otext': oText,
    'oslots': dict(valueType='str'),
    'book@en': langMetaData['en'],
}
for ft in set(nodeFeatures) | set(edgeFeatures):
    metaData.setdefault(
        ft, {}
    )['valueType'] = 'int' if ft in numFeatures else 'str'
    metaData[ft]['description'] = (
        specificMetaData[ft] if ft in specificMetaData else '?'
    )

# for cdir in (TEMP_DIR, TF_DIR):
os.makedirs(TF_DIR, exist_ok=True)

TF_DIR = os.path.expanduser('~/github/etcbc/linksyr/data/tf/blc')
print(f'Remove existing TF directory')
from shutil import rmtree
rmtree(TF_DIR)
print(f'Save TF dataset')
from tf.fabric import Fabric
TF = Fabric(locations=TF_DIR, silent=True)
TF.save(
    nodeFeatures=nodeFeatures,
    edgeFeatures=edgeFeatures,
    metaData=metaData
)



   8065 x slot
      1 x book
     49 x chapter
    344 x verse
    805 x lex
Compiling TF data
Building warp feature otype
9264 nodes
Filling in the nodes for features
Node features: title number word word_etcbc trailer lex sp nme emf nu gn st ls vbe vo vs vt ps pfm vbs vpm pfx otype book book@en
Edge features: oslots
Remove existing TF directory
Save TF dataset
   |     0.00s T book                 to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.00s T book@en              to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.00s T emf                  to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.01s T gn                   to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.02s T lex                  to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.01s T ls                   to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.01s T nme                  to /home/gdwarf/github/etcbc/linksyr/data/tf/blc
   |     0.01s T nu     